In [1]:
import numpy as np
import pandas as pd
import goodreads_api_client as gr
from bs4 import BeautifulSoup as bs
from bs4 import BeautifulSoup
import requests as req
import requests

from urllib.request import urlopen
import re

import time
import sys

### Getting popular science isbns

In [5]:
h = {'Authorization': '44538_7390dbc4419639e04527db4ebb786883'}
resp = req.get("https://api2.isbndb.com/book/9781934759486", headers=h)
print(resp.json())

{'book': {'publisher': 'Robert Reed Publishers', 'synopsys': "Math Jokes 4 Mathy Folks Is An Absolute Gem...---jim Rubillo Professor Emeritus, Bucks County Community College, Newtown, Pa The Jokes In This Book Are Well-chosen And Cover A Wide Spectrum, From Jokes For Kids To Jokes For Math Majors, From Corny To Thought-provoking---art Benjamin Professor And Mathemagician, Harvey Mudd College, Claremont, Ca This Is A Book That Every Math Teacher From Elementary School Through College Should Have In Their Classroom Library. Who Said Math Can't Be Funny?---victoria Miles, Middle Grades Math Teacher, Weymouth, Ma Patrick Vennebush Has Put Together The Most Comprehensive Set Of Mathematical Jokes I Have Ever Seen...if You Like Math And You Like Jokes---or If You Need A Joke To Liven Up An Otherwise Dull And Boring Lecture---then You Need To Buy This Book.---guy Brandenburg, Retired Teacher, Washington, Dc Math Nerds And Punsters Rejoice! This Is The Book You've Been Waiting For---your Perfe

In [6]:
n = 1
resp_list = []
for n in range(1,21):
    try:
        resp = req.get(f"https://api2.isbndb.com/subject/popular%20science?page={n}&pageSize=50", headers=h)
        print(n)
        n += 1
        resp_list.append(resp)
    except:
        print("No more books")
        break
        
len(resp_list)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


20

In [13]:
books = []
for resp in resp_list:
    books.extend(resp.json()['books'])
    
len(books)

1000

In [14]:
books[0]

{'image': 'https://images.isbndb.com/covers/28/25/9781125642825.jpg',
 'title_long': 'Manual-of-ship-model-making',
 'authors': ['Popular Science'],
 'title': 'Manual Of Ship Model Making',
 'isbn13': '9781125642825',
 'isbn': '1125642823'}

In [17]:
book_arr = []
for book in books:
    book_arr.append(book['isbn13'])

df = pd.DataFrame(book_arr, columns=['ISBN13'])

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN13  1000 non-null   object
dtypes: object(1)
memory usage: 7.9+ KB


### Getting reviews by isbn

In [193]:
user = gr.resources.User()

In [54]:
# link = hf.get_the_link(book_arr[0])
client = gr.Client(developer_key='fGkJVZsWA7At7eDPqUec4A')
book = client.Book.show_by_isbn(isbn=book_arr[1])

keys_wanted = ['id', 'title', 'isbn', 'reviews_widget']
reduced_book = {k:v for k, v in book.items() if k in keys_wanted}
widget = reduced_book['reviews_widget']

In [60]:
get_divs(book_arr[1])

/Users/dilkinn/Documents/GALVANIZE/CAPSTONE_3/PopularScienceBookRecommender/notebooks/src/helper_functions.py:9: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 9 of the file /Users/dilkinn/Documents/GALVANIZE/CAPSTONE_3/PopularScienceBookRecommender/notebooks/src/helper_functions.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(widget, 'html')


200
No more reviews


/Users/dilkinn/Documents/GALVANIZE/CAPSTONE_3/PopularScienceBookRecommender/notebooks/src/helper_functions.py:43: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 43 of the file /Users/dilkinn/Documents/GALVANIZE/CAPSTONE_3/PopularScienceBookRecommender/notebooks/src/helper_functions.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = bs(r.content, "html")


200


/Users/dilkinn/Documents/GALVANIZE/CAPSTONE_3/PopularScienceBookRecommender/notebooks/src/helper_functions.py:28: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 28 of the file /Users/dilkinn/Documents/GALVANIZE/CAPSTONE_3/PopularScienceBookRecommender/notebooks/src/helper_functions.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = bs(r.content, "html")


[]

In [61]:
book_arr[1]

'9780517654194'

In [64]:
link = hf.get_the_link(book_arr[3])
print(link)
pages = hf.get_all_pages(link)
print(pages)
divs = hf.div_reviews_all_pages(pages)

https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&format=html&isbn=1932273417&links=660&min_rating=&review_back=fff&stars=000&text=000
200
No more reviews
['https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&format=html&isbn=1932273417&links=660&min_rating=&review_back=fff&stars=000&text=000']
200


### Getting books and reviews directly from goodreads.

In [2]:
from importlib import reload  

# while True:
#     # Do some things.
#     if is_changed(foo):
#         foo = reload(foo)

hf = reload(hf)

NameError: name 'hf' is not defined

In [103]:
popular_science_link = 'https://www.goodreads.com/shelf/show/popular-science?page=1'
pages = [f'https://www.goodreads.com/shelf/show/popular-science?page={x}' for x in range(1, 26)]

In [121]:
pages

['https://www.goodreads.com/shelf/show/popular-science?page=1',
 'https://www.goodreads.com/shelf/show/popular-science?page=2',
 'https://www.goodreads.com/shelf/show/popular-science?page=3',
 'https://www.goodreads.com/shelf/show/popular-science?page=4',
 'https://www.goodreads.com/shelf/show/popular-science?page=5',
 'https://www.goodreads.com/shelf/show/popular-science?page=6',
 'https://www.goodreads.com/shelf/show/popular-science?page=7',
 'https://www.goodreads.com/shelf/show/popular-science?page=8',
 'https://www.goodreads.com/shelf/show/popular-science?page=9',
 'https://www.goodreads.com/shelf/show/popular-science?page=10',
 'https://www.goodreads.com/shelf/show/popular-science?page=11',
 'https://www.goodreads.com/shelf/show/popular-science?page=12',
 'https://www.goodreads.com/shelf/show/popular-science?page=13',
 'https://www.goodreads.com/shelf/show/popular-science?page=14',
 'https://www.goodreads.com/shelf/show/popular-science?page=15',
 'https://www.goodreads.com/shelf/

In [134]:
import time
b_links = []

In [202]:
page = pages[25]
r = requests.get(page)
soup = bs(r.content, 'html')
divs = soup.find_all('div', {"class": "elementList"})[:50]
links_for_books = ['http://goodreads.com/' + divs[i].find_all("a", {'class':"bookTitle"})[0].get('href') for i in range(50)]
b_links.extend(links_for_books)
# time.sleep(30)

IndexError: list index out of range

In [201]:
len(b_links)

1250

In [ ]:
## book_links

In [117]:
import time
def get_book_links(pages):
    """I have to add a time.sleep inbetween divs, not to hit the request limit."""
    b_links = []
    for page in pages:
        r = requests.get(link)
        soup = bs(r.content, 'html')
        divs = soup.find_all('div', {"class": "elementList"})[:50]
        page_b_links = ['http://goodreads.com/' + divs[i].find_all("a", {'class':"bookTitle"})[0].get('href') for i in range(len(divs))]
        b_links.extend(page_b_links)
        time.sleep(5)
    return b_links

In [206]:
all_books_links = b_links

In [204]:
len(all_books_links)

1250

# Get information about the books

In [217]:
with open('../data/books_links.txt', 'w') as f:
    for item in b_links:
        f.write(f"{item}\n")

In [3]:
b_l = []
with open('../data/books_links.txt', 'r') as f:
    lines = f.readlines()
    b_l.append(lines)
b_l = b_l[0]

In [4]:
b_l = [b_l[i].replace("\n","") for i in range(len(b_l))]

In [216]:
b_links

['http://goodreads.com//book/show/3869.A_Brief_History_of_Time',
 'http://goodreads.com//book/show/21.A_Short_History_of_Nearly_Everything',
 'http://goodreads.com//book/show/61535.The_Selfish_Gene',
 'http://goodreads.com//book/show/23692271-sapiens',
 'http://goodreads.com//book/show/55030.Cosmos',
 'http://goodreads.com//book/show/63697.The_Man_Who_Mistook_His_Wife_for_a_Hat_and_Other_Clinical_Tales',
 'http://goodreads.com//book/show/21413662-what-if-serious-scientific-answers-to-absurd-hypothetical-questions',
 'http://goodreads.com//book/show/1842.Guns_Germs_and_Steel',
 'http://goodreads.com//book/show/8049273-the-elegant-universe',
 'http://goodreads.com//book/show/3272165-bad-science',
 'http://goodreads.com//book/show/11468377-thinking-fast-and-slow',
 'http://goodreads.com//book/show/35167685-surely-you-re-joking-mr-feynman',
 'http://goodreads.com//book/show/6117055-the-greatest-show-on-earth',
 'http://goodreads.com//book/show/14743.The_God_Delusion',
 'http://goodreads.co

In [8]:
# for book_link in all_books_links:
book_link = b_l[0]    

In [10]:
def get_the_soup(book_link):
    r = requests.get(book_link)
    soup = bs(r.content, 'html.parser')
    return soup

def get_book_info(book_link, soup):
    isbn_str = soup.find_all('div',{"class":"infoBoxRowItem"})[1].get_text()

    isbn_str = isbn_str.replace('\n', '')
    isbn_str = isbn_str.replace('(', '')
    isbn_str = isbn_str.replace(')', '')
    isbn, isbn13 = isbn_str.strip().split()[0], isbn_str.strip().split()[2]

    title_str = soup.find_all('h1',{"id":"bookTitle"})[0].get_text()
    title_str = title_str.replace('\n', "").strip()

    original_title = soup.find_all('div',{"id":"bookDataBox"})[0].find_all('div',{'class':'infoBoxRowItem'})[0].get_text()

    avg_rating = soup.find_all('span',{"itemprop":"ratingValue"})[0].get_text()
    avg_rating = avg_rating.replace('\n', '').strip()
    avg_rating = float(avg_rating)

    author_link = soup.find_all('a',{"class":"authorName"})[0].get('href')
    spans = soup.find_all('a',{"class":"authorName"})[0].find_all('span',{'itemprop':'name'})

    authors_names = []
    for span in spans:
        authors_names.append(span.get_text())
    authors_names

    book_description = soup.find_all('div',{"id":"description"})[0].find_all('span')[1].get_text()
    
    dct = {'book_link': book_link, 'isbn': isbn, 'isbn13': isbn13, 
          'book_title' : title_str, 'original_title' : original_title,
          'avg_rating' : avg_rating, 'author_link': author_link,
          'author_names': authors_names, 'book_description': book_description}
    
    return dct

In [22]:
def get_reviews_from_soup(isbn, soup):
    divs = soup.find_all('div',{"id":"bookReviews"})[0].find_all('div',{'class':'friendReviews elementListBrown'})
    arr = []
    for div in divs:
        user_link = 'http://goodreads.com' + div.find_all('span',{'itemprop':'author'})[0].find('a').get('href')
        ranking = len(div.find_all('span', {'class':'staticStar p10'}))
        try:
            review = div.find_all('div', {'class':'reviewText stacked'})[0].find_all('span',{'class':"readable"})[0].find_all('span')[1].get_text()
        except:
            review = div.find_all('div', {'class':'reviewText stacked'})[0].find_all('span',{'class':"readable"})[0].find_all('span')[0].get_text()
        dct = {"isbn":isbn, 'user_link': user_link, 'ranking':ranking, "review": review}
        arr.append(dct)
    df = pd.DataFrame(arr)
    df.to_csv(f'../data/revs_{isbn}.csv')
    return df

In [1]:
books_arr = []
failed_books_list = []
for i, book_l in enumerate(b_l[:2]):
    sys.stdout.write("\r" + f'book n : {i}')
    sys.stdout.flush()
    try:
        soup = get_the_soup(book_l)
        dct = get_book_info(book_l, soup)
        isbn = dct['isbn']
        reviews = get_reviews_from_soup(isbn,soup)
        books_arr.append(dct)
        df = pd.DataFrame(books_arr)
        df.to_csv('../data/gr_books_df.csv')
    except:
        failed_books_list.append(book_l)
        continue

NameError: name 'b_l' is not defined